In [ ]:
%pip install librosa
%pip install matplotlib
%pip install panda

In [94]:
import glob
import os
import librosa
import numpy as np
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.svm import SVC
import sklearn.metrics as sm
import pandas as pd   
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier


In [56]:
######################### Helper Methods ################################

def backupFeatures(X, Y):
	X_df = pd.DataFrame(X)
	Y_df = pd.DataFrame(Y)
	X_df.to_csv("X_df_parallel.csv")
	Y_df.to_csv("Y_df_parallel.csv")

def featureExtraction(fileName):
	raw, rate = librosa.load(fileName)
	stft = np.abs(librosa.stft(raw))
	mfcc = np.mean(librosa.feature.mfcc(y=raw,sr=rate,n_mfcc=40).T, axis=0)
	chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=rate).T, axis=0)
	mel = np.mean(librosa.feature.melspectrogram(raw, sr=rate).T, axis=0)
	contrast = np.mean(librosa.feature.spectral_contrast(S=stft, sr=rate).T, axis=0)
	tonnetz = np.mean(librosa.feature.tonnetz(y=librosa.effects.harmonic(raw), sr=rate).T, axis=0)
	return mfcc, chroma, mel, contrast, tonnetz

def processSong(fn):
	mfcc, chroma, mel, contrast, tonnetz = featureExtraction(fn)
	tempFeatures = np.hstack([mfcc, chroma, mel, contrast, tonnetz])

	return tempFeatures

def processGenre(parentDirectory, subDir, i_low, i_high, fileExtension):
	features, labels = np.empty((0,193)), np.empty(0)
	i = i_low
	for fn in glob.glob(os.path.join(parentDirectory, subDir, fileExtension)):
		if i == i_high:
			break
		i += 1
		print(f"\tProcessing File: {fn}")
		mfcc, chroma, mel, contrast, tonnetz = featureExtraction(fn)
		tempFeatures = np.hstack([mfcc, chroma, mel, contrast, tonnetz])
		features = np.vstack([features, tempFeatures])
		# pop = 1, classical = 2, metal = 3, rock = 0
		global music_map
		labels = np.append(labels, music_map[subDir])
	return features, labels
	

# Takes parent directory name, subdirectories within parent directory, and file extension as input. 
def parseAudio(parentDirectory, subDirectories, i_low, i_high, 	fileExtension="*.wav"):
	
	inputVector = list()

	for subDir in subDirectories:
		inputVector.append([parentDirectory, subDir, i_low, i_high, fileExtension])
	
	with Pool(None) as p:
		results = p.starmap(processGenre, inputVector)
	
	features = np.empty((0,193))
	labels = np.empty(0)
	for result in results:
		features = np.vstack([features, result[0]])
		labels = np.append(labels, result[1])
	
	return np.array(features), np.array(labels, dtype=np.int)

music_map = {
	"blues": 0, "classical": 1, "country": 2, "disco": 3, "hiphop": 4, "jazz": 5, "metal": 6, "pop": 7, "reggae": 8, "rock": 9
}

subDirectories = ["blues", "classical", "country", "disco", "hiphop", "jazz", "metal", "pop", "reggae", "rock"]

X, Y = parseAudio("./GTZAN", subDirectories, 0, 10)



Processing Sub-Directory: blues
	Processing File: ./GTZAN\blues\blues.00000.wav
	Processing File: ./GTZAN\blues\blues.00001.wav
	Processing File: ./GTZAN\blues\blues.00002.wav
	Processing File: ./GTZAN\blues\blues.00003.wav
	Processing File: ./GTZAN\blues\blues.00004.wav
	Processing File: ./GTZAN\blues\blues.00005.wav
	Processing File: ./GTZAN\blues\blues.00006.wav
	Processing File: ./GTZAN\blues\blues.00007.wav
	Processing File: ./GTZAN\blues\blues.00008.wav
	Processing File: ./GTZAN\blues\blues.00009.wav
Processing Sub-Directory: classical
	Processing File: ./GTZAN\classical\classical.00000.wav
	Processing File: ./GTZAN\classical\classical.00001.wav
	Processing File: ./GTZAN\classical\classical.00002.wav
	Processing File: ./GTZAN\classical\classical.00003.wav
	Processing File: ./GTZAN\classical\classical.00004.wav
	Processing File: ./GTZAN\classical\classical.00005.wav
	Processing File: ./GTZAN\classical\classical.00006.wav
	Processing File: ./GTZAN\classical\classical.00007.wav
	Pro

C:\Users\AKSHAT~1\AppData\Local\Temp/ipykernel_21088/1659718494.py:43: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return np.array(features), np.array(labels, dtype=np.int)


In [57]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)

kmc = KMeans(n_clusters=10)
kmc.fit(X_train)
train_pred = kmc.predict(X_train)
test_pred = kmc.predict(X_test)
#################### Test Results ###################################

with open("XDataBig.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv, delimiter=',', newline='')
    csvWriter.writerows(X)

with open("YDataBig.csv","w+") as my_csv:
    csvWriter = csv.writer(my_csv, delimiter=',', newline='')
    csvWriter.writerow(Y)

TypeError: 'newline' is an invalid keyword argument for this function

In [114]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.5)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.1)

classifier = SVC(kernel='poly')
classifier.fit(X_train, Y_train)

y_pred = classifier.predict(X_test)

print(accuracy_score(Y_test, y_pred))


[2 8 0 8 7 3 2 5 4 6 0 4 5 1 5 7 8 8 9 6 6 5 4 9 7 2 3 8 1 1 7 9 3 7 6 4 3
 7 0 0 9 6 4 8 1 5 4 7 6 8]
[2 1 1 1 6 4 4 5 0 2 2 0 5 5 5 4 1 1 1 2 2 5 0 9 2 4 4 1 5 5 1 2 9 0 2 6 4
 5 5 5 2 2 2 1 5 5 4 1 2 1]
0.16


In [93]:
classifier = SVC(kernel='rbf')
classifier.fit(X_train, Y_train)

y_pred = classifier.predict(X_test)

print(accuracy_score(Y_test, y_pred))

0.22


In [115]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)

y_pred = classifier.predict(X_test)

print(accuracy_score(Y_test, y_pred))


0.28
